In [ ]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1) # Change the directory as appropriate

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

In [ ]:
def answer_zero():
    return df.iloc[0]
answer_zero(df)

In [ ]:
def answer_one():
        for country in df.index:
            if df.loc[country]['Gold'] == max(df['Gold']):
                return country
answer_one()

In [ ]:
def answer_two():
    gold_dif = []
    
    for country in df.index:
        gold_dif.append(df.loc[country]['Gold']-df.loc[country]['Gold.1'])
    gold_dif_df = pd.DataFrame({'countries': df.index, 'gold_medal_dif': gold_dif}, index=df.index)
    return gold_dif_df.loc[:]['gold_medal_dif'].idxmax(1)
answer_two()

In [ ]:
def answer_three():
    gold_dif = []
    total_gold_med = []
    for country in df.index:
        if df.loc[country]['Gold'] > 0 and df.loc[country]['Gold.1'] > 0:
            gold_dif.append(df.loc[country]['Gold']-df.loc[country]['Gold.1'])
            total_gold_med.append(df.loc[country]['Gold']+df.loc[country]['Gold.1'])
            ratio_gold = [x / y for x,y in zip(gold_dif, total_gold_med)]
            pos_ratio_gold = [abs(number) for number in ratio_gold]
    gold_df = pd.DataFrame({'gold_medal_dif': gold_dif, 
                             'Total_medal': total_gold_med, 'Ratio': pos_ratio_gold}, 
                           index = [country for country in df.index if df.loc[country]['Gold'] > 0 and 
                             df.loc[country]['Gold.1'] > 0])
    return gold_df.loc[:]['Ratio'].idxmax(1)
answer_three()

In [ ]:
def answer_four():
    counts=[]
    count_it=0
    for country in df.index:
        count_it=0
        if df.loc[country]['Gold.2'] > 0:
            count_it = 3 * df.loc[country]['Gold.2']
        else:
            count_it = 0
        if df.loc[country]['Silver.2'] > 0:
            count_it = count_it + (2 * df.loc[country]['Silver.2'])
        else:
            count_it = 0
        if df.loc[country]['Bronze.2'] > 0:
            count_it = count_it + (df.loc[country]['Bronze.2'])
        else:
            count_it = 0
        counts.append(count_it)   
    Points_df = pd.DataFrame({'Points': counts}, index= df.index)
    return Points_df
answer_four()

In [ ]:
def answer_five():
    census_df = pd.read_csv('census.csv', encoding = 'ISO-8859-1') # Change the directory as appropriate
    census_df = census_df.set_index('STNAME')
    sum_across_state = census_df.sum(level=0)
    sum_counties = census_df.sum(level=0)['COUNTY']
    return sum_counties.idxmax()
answer_five()

In [ ]:
def answer_six():
    df = census_df[census_df['SUMLEV'] == 50]
    df1 = census_df.sort_values(['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3).copy()
    df2 = df1.reset_index().groupby("STNAME").sum().sort_values(['CENSUS2010POP'],ascending=False).head(3).copy()
    return list(df2.index.values)
answer_six()

In [ ]:
def answer_seven():
    df = census_df[census_df['SUMLEV'] == 50]
    df['STDEV'] = df[['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012',
                     'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']].std(axis=1)
    return df.loc[df['STDEV'].idxmax()]['CTYNAME']
answer_seven()

In [ ]:
def answer_eight():
    df=census_df[census_df['SUMLEV'] == 50]
    region1 = df['REGION'] == 1
    region2 = df['REGION'] == 2
    county_name = df['CTYNAME'].str.startswith('Washington')
    pop_estim = df['POPESTIMATE2015'] > df['POPESTIMATE2014']
    df = df[(region2|region1)&county_name&pop_estim]
    return df[['STNAME', 'CTYNAME']]
answer_eight()